# Zero-Coupon Bond Using Marlowe\'s Command-Line Interface

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

[A video works through this Jupyter notebook.](https://youtu.be/ELc72BKf7ec)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In this demonsration we use Marlowe\'s command-line interface, `marlowe-cli`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use addresses.

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](../../images/01-zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Address "$LENDER_ADDR")
            (Address "$LENDER_ADDR")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Address "$LENDER_ADDR")
            (Party (Address "$BORROWER_ADDR"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Address "$BORROWER_ADDR")
                        (Address "$BORROWER_ADDR")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Address "$BORROWER_ADDR")
                        (Party (Address "$LENDER_ADDR"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close 
```

## Preliminaries

See [Preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this tutorial.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [ ]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

Make sure you've also [setup and funded](../../setup/01-setup-keys.ipynb) the different parties

- Lender
    - \$KEYS/lender.address: Cardano address for the lender
    - \$KEYS/lender.skey: location of signing key file for the lender
- Borrower
    - \$KEYS/borrower.address: Cardano address for the borrower
    - \$KEYS/borrower.skey: location of signing key file for the borrower

### Lender address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
LENDER_SKEY=$KEYS/lender.skey
LENDER_ADDR=$(cat $KEYS/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

Check that the lender has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $LENDER_ADDR

### Borrower address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
BORROWER_SKEY=$KEYS/borrower.skey
BORROWER_ADDR=$(cat $KEYS/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

Check that the borrower has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $BORROWER_ADDR

## Design the contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe.iohk.io//) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Here we generate the contract using Marlowe CLI.

First, set the loan's principal to 80 ada and its interest to 5 ada.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace

PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))

echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

Next find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [ ]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [ ]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo LENDER_DEADLINE = "$LENDER_DEADLINE" POSIX milliseconds = "`date -d @$((LENDER_DEADLINE / SECOND))`"
echo BORROWER_DEADLINE = "$BORROWER_DEADLINE" POSIX milliseconds = "`date -d @$((BORROWER_DEADLINE / SECOND))`"

Now create the JSON file for the contract, `zcb-contract.json`.

In [ ]:
# Create the contract and initial state
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file zcb-state.json

# Show the size of the files
du -hs zcb-contract.json zcb-state.json

The various command-line options are described by the help system.

In [ ]:
marlowe-cli template zcb --help

## Examine the contract

View the contract file as YAML.

In [ ]:
json2yaml zcb-contract.json

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1. Create the Contract

Marlowe CLI\'s command `marlowe-cli run initialize` will build the creation information for a Marlowe contract. We provide it the JSON files containing the contract and initial state. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [ ]:
marlowe-cli run initialize --help

In [ ]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file zcb-contract.json \
  --state-file zcb-state.json \
  --out-file marlowe-1.json

du -hs marlowe-1.json

We now use Marlowe CLI\'s `marlowe-cli run auto-execute` command to construct and submit the creation transaction.

In [ ]:
marlowe-cli run auto-execute --help

In [ ]:
echo "submiting transaction..."
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file marlowe-1.json \
  --change-address "$LENDER_ADDR" \
  --required-signer "$LENDER_SKEY" \
  --out-file tx-1.signed \
  --submit 600s \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_1

One can also examine the contract's UTxO using `cardano-cli`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe CLI\'s `marlowe-cli run prepare` command. The lender is providing the funding for and receiving the change from this transaction, so we provide their address.

In [ ]:
marlowe-cli run prepare --help

In [ ]:
marlowe-cli run prepare \
  --deposit-account "$LENDER_ADDR" \
  --deposit-party "$LENDER_ADDR" \
  --deposit-amount "$PRINCIPAL" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-1.json \
  --out-file marlowe-2.json

du -hs marlowe-2.json 

Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [ ]:
echo "submiting transaction..."
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file marlowe-1.json \
  --marlowe-out-file marlowe-2.json \
  --change-address "$LENDER_ADDR" \
  --required-signer "$LENDER_SKEY" \
  --out-file tx-2.signed \
  --submit 600s \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_2

One can see that the lender has 82 ada less than originally. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

The borrower has an additional 80 ada (the loan's principal) now.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

The Marlowe contract still has the 2 ada from its creation.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

## Transaction 3. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

In [ ]:
marlowe-cli run prepare \
  --deposit-account "$BORROWER_ADDR" \
  --deposit-party "$BORROWER_ADDR" \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-2.json \
  --out-file marlowe-3.json
du -hs marlowe-3.json

Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [ ]:
echo "submiting transaction..."
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file marlowe-2.json \
  --marlowe-out-file marlowe-3.json \
  --change-address "$BORROWER_ADDR" \
  --required-signer "$BORROWER_SKEY" \
  --out-file tx-3.signed \
  --submit 600s \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_3

One can see that the lender received back the 80 ada of principal and the 2 ada deposited when the contract was created, along with the additional 5 ada of interest, totallying 87 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

The borrower now has about 5 ada (the loan's interest) less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

The Marlowe contract has closed, so there is no output to its script address.